# Relatório de felicidade do mundo

Objetivo: Qual é o melhor país para viver feliz?

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from ortools.linear_solver import pywraplp

# Carregando os dois arquivos CSV para analisar
dataset_1 = pd.read_csv('/kaggle/input/world-happiness-report-2021/world-happiness-report-2021.csv')
dataset_2 = pd.read_csv('/kaggle/input/world-happiness-report-2021/world-happiness-report.csv')

# Primeira Etapa: ETL

As fontes de dados são duas, distintas mas com colunas semelhantes. 

A primeira fonte é um relatório de felicidade do ano vingente (2021). A segunda fonte é a série histórica de cada país.

Meu objetivo é juntar as duas fontes e fazer uma média de cada país.

In [ ]:
# Ajustando o nome das colunas
dataset_1 = dataset_1.rename(columns={
    'Country name': 'country_name',
    'Ladder score': 'ladder_score',
    'Social support': 'social_support',
    'Logged GDP per capita': 'logged_gdp_per_capita',
    'Healthy life expectancy': 'healthy_life_expectancy',
    'Freedom to make life choices': 'freedom_make_life_choices',
    'Generosity': 'generosity',
    'Perceptions of corruption': 'perceptions_corruption',
})

# Selecionando o subconjunto somente com as colunas necessárias
dataset_1 = dataset_1[['country_name', 'ladder_score', 'social_support', 'logged_gdp_per_capita', 'healthy_life_expectancy', 'freedom_make_life_choices', 'generosity', 'perceptions_corruption']]
dataset_1.head()

In [ ]:
# Ajustando o nome das colunas
dataset_2 = dataset_2.rename(columns={
    'Country name': 'country_name',
    'Life Ladder': 'ladder_score',
    'Log GDP per capita': 'logged_gdp_per_capita',
    'Social support': 'social_support',
    'Healthy life expectancy at birth': 'healthy_life_expectancy',
    'Freedom to make life choices': 'freedom_make_life_choices',
    'Generosity': 'generosity',
    'Perceptions of corruption': 'perceptions_corruption',
})

# Selecionando o subconjunto somente com as colunas necessárias
dataset_2 = dataset_2[['country_name', 'ladder_score', 'social_support', 'logged_gdp_per_capita', 'healthy_life_expectancy', 'freedom_make_life_choices', 'generosity', 'perceptions_corruption']]
dataset_2.head()

In [ ]:
# Juntando os dois datasets
dataset = pd.merge(dataset_1, dataset_2, how='outer')
dataset.fillna(0, inplace=True)
dataset = dataset.groupby(['country_name']).median()
dataset.reset_index(inplace=True)
dataset.head()

## Segunta Etapa: Aplicar Modelo

Com base na pergunta proposta no início, o modelo mais adequado seria Prescritivo, aplicando uma otimização com base em variáveis definidas. 
O modelo Prescritivo é usado quando se precisa encontrar a melhor decisão ou o melhor caminho com base nos dados que você possui. No contexto proposto, precisamos achar o melhor país para se viver feliz e por isso o modelo prescritivo melhor se aplica.

Com base no site: https://www.trackinghappiness.com/happiness-index-2018/, foram definidas as algumas variáveis, levando em conta cada aspecto que pode influenciar a felicidade em um país. Cada aspecto da vida pode influenciar diretamente na felicidade dos cidadão, como a percepção de corrupção, expectativo de vida melhor, liberdade para trilhar seu próprio caminho. Cada aspecto desse foi levado em conta e com base nos textos encontrados, foram definidas as seguintes variáveis:

In [ ]:
# Gerando um gráfico comparativo da distribuição de cada aspecto
dataset.hist(figsize=(20,20))

In [ ]:
# Variávies estipuladas com base nos gráficos acima para realização da otimização
variables = [
    ['ladder_score', 8],
    ['social_support', 1],
    ['logged_gdp_per_capita', 12],
    ['healthy_life_expectancy', 75],
    ['freedom_make_life_choices', 0.9],
    ['generosity', 0],
    ['perceptions_corruption', 0.2],
]

In [ ]:
data = dataset.to_numpy()

solver = pywraplp.Solver('SolveStigler', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

countries = [solver.NumVar(0.0, solver.infinity(), item[0]) for item in data]

objective = solver.Objective()

for country in countries:
    objective.SetCoefficient(country, 1)
objective.SetMinimization()

constraints = []

for i, variable in enumerate(variables):
    constraints.append(solver.Constraint(variable[1], solver.infinity()))
    for j, item in enumerate(data):
        constraints[i].SetCoefficient(countries[j], item[i + 1])
        
print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())

In [ ]:
status = solver.Solve()

if status != pywraplp.Solver.OPTIMAL:
    print("The problem does not have an optimal solution!")
    exit(1)
    
variables_result = []

for i, country in enumerate(countries):
    if country.solution_value() > 0.0:
        variables_result.append([country.name(), country.solution_value()])

result = pd.DataFrame(variables_result, columns=['Country', 'Result'])

print('Objective value:', objective.Value())
print('Problem solved in', solver.wall_time(), 'milliseconds')
print('Problem solved in', solver.iterations(), 'iterations')

In [ ]:
# Gerando um gráfico estilo Pizza para fins de comparação dos resultados
labels = result['Country']
fig1, ax1 = plt.subplots()
ax1.pie(result['Result'], labels=labels, autopct='%1.1f%%', shadow=True, startangle=90)
plt.show()

In [ ]:
# Gerando um gráfico estilo barras para fins de comparação dos resultados
plt.bar(labels, result['Result'])
plt.title("Comparacao dos Resultados")
plt.show()

## Conclusão

Com base nos resultados da aplicação do modelo de Optimization e nos gráficos gerados, o melhor país para viver feliz é Switzerland, sem também deixar de lado Luxembourg que também foi detectada como uma possibilidade viável.

O modelo prescritivo foi aplicado juntamente com uma otimização dos dados para melhores análises.